In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-client:3.3.4,org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.1.0 pyspark-shell'

In [ ]:
app_name = "CeroFugas"
master = "local[*]"

conf = SparkConf().setAppName(app_name)\
    .setMaster(master)

# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider')
# conf.set('spark.hadoop.fs.s3a.access.key', <access_key>)
# conf.set('spark.hadoop.fs.s3a.secret.key', <secret_key>)
# conf.set('spark.hadoop.fs.s3a.session.token', <token>)

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName(app_name).getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
# Set the MinIO access key, secret key, endpoint, and other configurations
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "GVTZ4kb69J0Gop6YaWSf")
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "WDkCNivmHhMRrBPgOWRIHZSHb5iMNyzBJgSgFk4f")
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", 'org.apache.hadoop.fs.s3a.S3AFileSystem')
sc._jsc.hadoopConfiguration().set('fs.s3.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
# Read a J

df = spark.read.parquet("s3a://cero-fugas/reportes_agua_2024_01.parquet")

In [ ]:
spark = SparkSession.builder.appName(app_name).getOrCreate()

spark.sparkContext._jsc\
     .hadoopConfiguration().set("fs.s3a.access.key", "GVTZ4kb69J0Gop6YaWSf")
spark.sparkContext._jsc\
     .hadoopConfiguration().set("fs.s3a.secret.key", "WDkCNivmHhMRrBPgOWRIHZSHb5iMNyzBJgSgFk4f")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc\
      .hadoopConfiguration().set("fs.s3a.path.style.access", "true")


df = spark.read.parquet("s3a://cero-fugas/reportes_agua_2024_01.parquet")
df.show()

In [ ]:
from pyspark import SparkConf 
from pyspark.sql import SparkSession
import os

# Configurar path de jars de Hadoop
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-aws:3.3.5 pyspark-shell'
conf = SparkConf()
conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-common:3.3.5,org.apache.hadoop:hadoop-aws:3.3.5")
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", "GVTZ4kb69J0Gop6YaWSf")
hadoop_conf.set("fs.s3a.secret.key", "WDkCNivmHhMRrBPgOWRIHZSHb5iMNyzBJgSgFk4f")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.path.style.access", "true")

In [ ]:
df = spark.read.format("parquet").load("s3a://cero-fugas/reportes_agua_2024_01.parquet")

In [ ]:
# df = spark.read.format("parquet").load("s3a://cero-fugas/reportes_agua_2024_01.parquet")
df = spark.read.format("parquet").load("../DATASETS/reportes_agua_2024_01.parquet")
df.show()

In [ ]:
from pyspark import SparkConf 
from pyspark.sql import SparkSession
import os

# Configurar variables de entorno para los JARs
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-aws:3.3.5 pyspark-shell'

# Crear configuración de Spark
conf = SparkConf()
conf.set("spark.jars.packages", 
         "org.apache.hadoop:hadoop-common:3.3.5,"
         "org.apache.hadoop:hadoop-aws:3.3.5")
conf.set("spark.driver.extraClassPath", "hadoop-aws:3.3.5:aws-java-sdk-bundle-1.12.779.jar")
conf.set("spark.executor.extraClassPath", "hadoop-aws:3.3.5:aws-java-sdk-bundle-1.12.779.jar")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Crear sesión de Spark
spark = SparkSession.builder \
    .config(conf=conf) \
    .config("spark.hadoop.fs.s3a.endpoint", "http://tu-endpoint-minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "GVTZ4kb69J0Gop6YaWSf") \
    .config("spark.hadoop.fs.s3a.secret.key", "WDkCNivmHhMRrBPgOWRIHZSHb5iMNyzBJgSgFk4f") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

# Leer el archivo
df = spark.read.format("parquet").load("s3a://cero-fugas/reportes_agua_2024_01.parquet")
df.show()

In [ ]:
import os
import sys

# Verificar variables de entorno
print("JAVA_HOME:", os.environ.get('JAVA_HOME'))
print("SPARK_HOME:", os.environ.get('SPARK_HOME'))
print("HADOOP_HOME:", os.environ.get('HADOOP_HOME'))
print("Python executable:", sys.executable)

In [ ]:
import os
import findspark
from pyspark.sql import SparkSession

# Configurar FindSpark si es necesario
findspark.init()

# Configurar variables de entorno
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
# Configuración de paquetes más simple
# Configuración de paquetes más completa
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--packages com.amazonaws:aws-java-sdk-bundle:1.12.779,'
    'org.apache.hadoop:hadoop-aws:3.3.5,'
    'org.apache.hadoop:hadoop-client:3.3.5 '
    'pyspark-shell'
)

# Ubicación de JARs adicionales
hadoop_jars_path = r"E:\Spark\spark-3.5.3-bin-hadoop3\jars"

# Configuración de Spark
spark = SparkSession.builder \
    .appName("MinIO Parquet Reader") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "GVTZ4kb69J0Gop6YaWSf") \
    .config("spark.hadoop.fs.s3a.secret.key", "WDkCNivmHhMRrBPgOWRIHZSHb5iMNyzBJgSgFk4f") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.driver.extraClassPath", f"{hadoop_jars_path}/*") \
    .config("spark.executor.extraClassPath", f"{hadoop_jars_path}/*") \
    .getOrCreate()


In [ ]:
try:
    # Leer el archivo
    df = spark.read.format("parquet").load("s3a://cero-fugas/reportes_agua_2024_01.parquet")
    df.show()
except Exception as e:
    print(f"Error al leer el archivo: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
from pyspark import SparkConf 
from pyspark.sql import SparkSession
import os

# Ruta a tu instalación de Spark
SPARK_HOME = r"E:\Spark\spark-3.5.3-bin-hadoop3"

# Configurar variables de entorno
os.environ['PYSPARK_SUBMIT_ARGS'] = (
    '--packages com.amazonaws:aws-java-sdk-bundle:1.12.262,'
    'org.apache.hadoop:hadoop-aws:3.3.5,'
    'com.amazonaws:aws-java-sdk-bundle:1.12.779,'
    'org.apache.hadoop:hadoop-client:3.3.5 pyspark-shell'
)

# Configuración de Spark
spark = SparkSession.builder \
    .appName("MinIO Parquet Reader") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "GVTZ4kb69J0Gop6YaWSf") \
    .config("spark.hadoop.fs.s3a.secret.key", "WDkCNivmHhMRrBPgOWRIHZSHb5iMNyzBJgSgFk4f") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.driver.extraClassPath", f"{SPARK_HOME}/jars/*") \
    .config("spark.executor.extraClassPath", f"{SPARK_HOME}/jars/*") \
    .getOrCreate()

try:
    # Leer el archivo
    df = spark.read.format("parquet").load("s3a://cero-fugas/reportes_agua_2024_01.parquet")
    df.show()
except Exception as e:
    print(f"Error al leer el archivo: {e}")
    import traceback
    traceback.print_exc()

Remote

In [1]:
from pyspark.sql import SparkSession

# Configuración para conexión a cluster remoto
spark = SparkSession.builder \
    .master("spark://192.168.47.100:7077") \
    .appName("MinIO Parquet Reader") \
    .config("spark.submit.deployMode", "client") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "GVTZ4kb69J0Gop6YaWSf") \
    .config("spark.hadoop.fs.s3a.secret.key", "WDkCNivmHhMRrBPgOWRIHZSHb5iMNyzBJgSgFk4f") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.jars.packages", 
        "com.amazonaws:aws-java-sdk-bundle:1.12.779," + 
        "org.apache.hadoop:hadoop-aws:3.3.5") \
    .getOrCreate()


In [2]:
try:
    # Leer el archivo
    df = spark.read.format("parquet").load("s3a://cero-fugas/reportes_agua_2024_01.parquet")
    df.show()
except Exception as e:
    print(f"Error al leer el archivo: {e}")
    import traceback
    traceback.print_exc()

Error al leer el archivo: An error occurred while calling o43.read.
: java.lang.IllegalStateException: LiveListenerBus is stopped.
	at org.apache.spark.scheduler.LiveListenerBus.addToQueue(LiveListenerBus.scala:92)
	at org.apache.spark.scheduler.LiveListenerBus.addToStatusQueue(LiveListenerBus.scala:75)
	at org.apache.spark.sql.internal.SharedState.<init>(SharedState.scala:115)
	at org.apache.spark.sql.SparkSession.$anonfun$sharedState$1(SparkSession.scala:143)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sharedState$lzycompute(SparkSession.scala:143)
	at org.apache.spark.sql.SparkSession.sharedState(SparkSession.scala:142)
	at org.apache.spark.sql.SparkSession.$anonfun$sessionState$2(SparkSession.scala:162)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.SparkSession.sessionState$lzycompute(SparkSession.scala:160)
	at org.apache.spark.sql.SparkSession.sessionState(SparkSession.scala:157)
	at org.apache.spark.sql.DataFrameRead

Traceback (most recent call last):
  File "C:\Users\kiramishima\AppData\Local\Temp\ipykernel_12200\470218523.py", line 3, in <module>
    df = spark.read.format("parquet").load("s3a://cero-fugas/reportes_agua_2024_01.parquet")
         ^^^^^^^^^^
  File "E:\Spark\spark-3.5.3-bin-hadoop3\python\pyspark\sql\session.py", line 1706, in read
    return DataFrameReader(self)
           ^^^^^^^^^^^^^^^^^^^^^
  File "E:\Spark\spark-3.5.3-bin-hadoop3\python\pyspark\sql\readwriter.py", line 70, in __init__
    self._jreader = spark._jsparkSession.read()
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kiramishima\.conda\envs\data_engineer\Lib\site-packages\py4j\java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "E:\Spark\spark-3.5.3-bin-hadoop3\python\pyspark\errors\exceptions\captured.py", line 179, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^
  File "c:\Users\kiramishima\.conda\envs\data_engineer\Li